In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, Dataset
import torchvision 
import torchvision.models as models
import torchvision.transforms as T
import torch.optim as optim
from torchvision.datasets import ImageFolder
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches
from pathlib import Path
import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform

In [2]:
class myDataset(Dataset):
    def __init__(self, main_dir, train=True,mask=False, transform=None):
        self.split = "train" if train else "test"
        self.dset_dir = Path(main_dir)/self.split
        self.transform = transform
        self.files = []
        self.mask=mask
        self.df_mask=pd.DataFrame()
        folders = sorted(os.listdir(self.dset_dir))
        
        if self.split=="train" :
            if  mask:
                self.df_mask=pd.read_csv(main_dir+"train.csv")
            for folder in folders:
                class_idx= folders.index(folder)
                folder_dir = self.dset_dir/folder
                files = os.listdir(folder_dir)
                self.files += [{"mask":folder+"/"+x,"file": folder_dir/x, "class": class_idx+1} for x in files]
                
        else:
            self.file=folders
            for file in folders:
                 self.files.append(self.dset_dir/file)
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        
        if self.split == "train":
            item = self.files[i]
            file = item['file']
            
            # reading the images and converting them to correct size and color    
            img = cv2.imread(str(file))
            img_res = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
            img_res /= 255.0
            wt = 350
            ht = 350
            
            # recover bounding boxes
            name_file_img=item['mask']
            mask_data=self.df_mask[self.df_mask["image"]==name_file_img]
            xmin=int(mask_data["x1"])
            ymin=int(mask_data["y1"])
            xmax=int(mask_data["x2"])
            ymax=int(mask_data["y2"])
            
            # resize bounding boxes
            boxes = []
            xmin_corr = xmin+1
            xmax_corr = xmax-1
            ymin_corr = ymin+1
            ymax_corr = ymax-1
            
            boxes.append([xmin_corr, ymin_corr, xmax_corr, ymax_corr])
            # convert boxes into a torch.Tensor
            boxes = torch.as_tensor(boxes, dtype=torch.int64)
            
            # getting the areas of the boxes
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            

            # suppose all instances are not crowd
            iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
           
            
            labels = torch.tensor(item['class'],dtype=torch.int64)
            labels=labels.unsqueeze(0)
            
            target = {}
            target["boxes"] = boxes
            target["labels"] = labels
            target["area"] = area
            target["iscrowd"] = iscrowd
            # image_id
            
            target["image_id"] = torch.tensor([i])
            
            
            if self.transform:
            
                sample = {'image' : img_res,
                          'bboxes' : target['boxes'],
                          'labels' : labels
                         }
               
                sample = self.transform(**sample)
                img_res = sample['image']
               
                target['boxes'] = torch.as_tensor((sample['bboxes']),dtype=torch.int64)
                
                
               
            return img_res, target
        else:
            file = self.files[i]
            
            
            # reading the images and converting them to correct size and color    
            img = cv2.imread(str(file))
            img_res = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
            # diving by 255
            img_res /= 255.0
            
            if self.transform:
                sample = {
                    'image': img_res,
                }
                sample = self.transform(**sample)
                image = sample['image']
            return image,self.file[i]
        

In [3]:
main_dir = "../input/aiunict-2023/"

In [10]:
def train_transf():
    return A.Compose([
        A.Flip(0.5),
        A.Blur(blur_limit=(3, 6), p=0.8),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def test_transf():
    return A.Compose([
        ToTensorV2(p=1.0)
    ])

def collate_fn(batch):
    return tuple(zip(*batch))

In [11]:
train_set = myDataset(main_dir, train=True, mask=True, transform=train_transf())
train_loader= DataLoader(train_set, batch_size=8, shuffle=True, num_workers=4, collate_fn=collate_fn)
test_set = myDataset(main_dir, train=False, transform= test_transf())

In [ ]:
# plot the images with their bounding boxes
images, targets = next(iter(train_loader))
fig, axs = plt.subplots(2, 4, figsize=(15, 10))
for i in range(8):
    ax = axs[i//4][i%4]
    img = images[i].permute(1, 2, 0).numpy()
    ax.imshow(img)
    boxes = targets[i]["boxes"].numpy()
    for box in boxes:
        xmin, ymin, xmax, ymax = box
        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
plt.show()

In [6]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
fcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(pretrained = True)
in_features = fcnn.roi_heads.box_predictor.cls_score.in_features
num_classes = 9
fcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_v2_coco-dd69338a.pth


  0%|          | 0.00/167M [00:00<?, ?B/s]

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fcnn.to(device)
params = [p for p in fcnn.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

In [8]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
loss_hist = Averager()
itr = 1
lossHistoryiter = []
lossHistoryepoch = []
true_lab=[]
import time
start = time.time()

for epoch in range(5):
    loss_hist.reset()
    fcnn.train()
    for images, targets in train_loader:
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = fcnn(images, targets)  
        
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        
        loss_hist.send(loss_value)
        lossHistoryiter.append(loss_value)
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()
        
    lossHistoryepoch.append(loss_hist.value)
    print(f"Epoch #{epoch} loss: {loss_hist.value}")
    
torch.save(fcnn, "/kaggle/working/model.pth")
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time taken to Train the model :{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [12]:
def apply_nms(orig_prediction, iou_thresh=0.3):
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

In [13]:
image_id=[]
image_class=[]
for idx in range(test_set.__len__()):
    img,name_file = test_set[idx]
    fcnn.eval()
    with torch.no_grad():
        prediction = fcnn([img.to(device)])[0]
    nms_prediction = apply_nms(prediction, iou_thresh=0.2)
    pred=nms_prediction['labels'].cpu().numpy()[0]
    
    image_id.append(name_file)
    image_class.append(pred-1)
    
d = {'image': image_id, 'class': image_class}
df = pd.DataFrame(data=d)

In [14]:
df.to_csv("submissiontry3.csv",index=False)

In [16]:
os.remove("/kaggle/working/model-1.pth")